# Repressilator split between 2 strains

This repressilator is split between 2 strains - one stops producing red fluorescent protein and C14 in response to C4. Second strain has yellow fluorescence and tetR synthesis repressed by C14, and tetR in turn represses cyan fluorescence and C4 synthesis.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return 1 #lc.gompertz_growth_rate(t, 0.005, 0.5, 1, 1)

def biomass(t):
    return 0.5 #lc.gompertz(t, 0.005, 0.5, 1, 1)

## Strain 1 (one node)

In [ ]:
k2 = [1, 10, 100, 1000]
alpha = [10**-8, 10**-7, 10**-6, 10**-5, 10**-4, 10**-3, 10**-2, 0.1]
flow = [10, 100, 1000]

for f in flow:
        for a in alpha:
                for k in k2:
                        """ strain 1 """
                        gn1 = lc.GeneticNetwork()
                        metab1 = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
                        strain_1 = lc.Strain('Strain 1', gn1, metab1)
                        # Transcription factors
                        laci_1 = lc.Regulator(name='LacI', degradation_rate=1, init_concentration=10**-7)

                        # Signalling molecules
                        c4_1 = lc.Regulator(name='C4', permeability_const=100, degradation_rate=1)
                        c14_1 = lc.Regulator(name='C14', permeability_const=100, degradation_rate=1)

                        gn1.add_regulator([laci_1, c4_1, c14_1])

                        cini = lc.Regulator(name='CinI', degradation_rate=1)
                        gn1.add_regulator(cini)

                        mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
                        gn1.add_reporter(mcherry)
                        c4_laci = lc.Hill1(name='Prhl', input=c4_1, output=[laci_1,mcherry], alpha=[0, a], K=10**-5, n=1.255)
                        laci_not_cini = lc.Hill1(name='Plac', input=laci_1, output=cini, alpha=[10**-4, 0], K=10**-8, n=2)
                        cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14_1, k2=k)

                        gn1.add_operator([c4_laci, laci_not_cini])
                        gn1.add_impactor(cini_c14)

                        """ strain 2 """

                        gn2 = lc.GeneticNetwork()
                        metab2 = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
                        strain_2 = lc.Strain('Strain 2', gn2, metab2)

                        # Transcription factors
                        laci_2 = lc.Regulator(name='LacI', degradation_rate=1)
                        tetr_2 = lc.Regulator(name='TetR', degradation_rate=1)

                        # Signalling molecules
                        c4_2 = lc.Regulator(name='C4', permeability_const=100, degradation_rate=1)
                        c14_2 = lc.Regulator(name='C14', permeability_const=100, degradation_rate=1)

                        gn2.add_regulator([tetr_2, laci_2, c4_2, c14_2])

                        rhli = lc.Regulator(name='RhlI', degradation_rate=1)
                        gn2.add_regulator(rhli)

                        cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
                        mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

                        gn2.add_reporter([cyan, mven])

                        # One node
                        c14_laci = lc.Hill1(name='Pcin', input=c14_2, output=[laci_2, mven], alpha=[0, a], K=10**-5, n=1.255)
                        laci_not_tetr = lc.Hill1(name='Plac', input=laci_2, output=[tetr_2, cyan], alpha=[10**-4, 0], K=10**-8, n=2)
                        # Second node
                        tetr_not_c4 = lc.Hill1(name='Ptet', input=tetr_2, output=rhli, alpha=[10**-4, 0], K=10**-8, n=2)
                        rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4_2, k2=k)

                        gn2.add_operator([c14_laci, laci_not_tetr, tetr_not_c4])
                        gn2.add_impactor(rhli_c4)

                        """ set up sample, assay and simulate """
                        
                        sample = lc.Sample(strain=[strain_1, strain_2])
                        sample.set_extracel_flow('C4', f)
                        sample.set_extracel_flow('C14', f)
                        assay = lc.Assay([sample], 
                                        n_measurements=100, 
                                        interval=0.24,
                                        name='Extended repressilator',
                                        description='Simulated expression of extended repressilator strain'
                                        )

                        assay.run(substeps=50, mode="track_all")

                        m = assay.measurements
                        fig, ax = plt.subplots()
                        fig1, ax1 = plt.subplots()
                        fig2, ax2 = plt.subplots()
                        fig3, ax3 = plt.subplots()
                        m[m.Signal=='mCherry in Strain 1'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
                        m[m.Signal=='LacI in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax1)
                        m[m.Signal=='C4 in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax2)
                        m[m.Signal=='C14 in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax2)
                        m[m.Signal=='CinI in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax3)
                        ax.legend(['mCherry'])
                        ax.set_xlabel('Time (h)')
                        ax.set_ylabel('Concentration (M)')
                        ax1.legend(['LacI in S1'])
                        ax1.set_xlabel('Time (h)')
                        ax1.set_ylabel('Concentration (M)')
                        ax2.legend(['C4 in S1', 'C14 in S1'])
                        ax2.set_xlabel('Time (h)')
                        ax2.set_ylabel('Concentration (M)')
                        ax3.legend(['CinI'])
                        ax3.set_xlabel('Time (h)')
                        ax3.set_ylabel('Concentration (M)')

                        fig5, ax5 = plt.subplots()
                        fig6, ax6 = plt.subplots()
                        fig7, ax7 = plt.subplots()
                        fig4, ax4 = plt.subplots()
                        m[m.Signal=='CFP in Strain 2'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax4)
                        m[m.Signal=='mVenus in Strain 2'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax4)
                        m[m.Signal=='LacI in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax5)
                        m[m.Signal=='TetR in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax5)
                        m[m.Signal=='C4 in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax6)
                        m[m.Signal=='C14 in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax6)
                        m[m.Signal=='RhlI in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax7)
                        ax4.legend(['CFP', 'mVenus'])
                        ax4.set_xlabel('Time (h)')
                        ax4.set_ylabel('Concentration (M)')
                        ax5.legend(['LacI in S2', 'TetR'])
                        ax5.set_xlabel('Time (h)')
                        ax5.set_ylabel('Concentration (M)')
                        ax6.legend(['C4 in S2', 'C14 in S2'])
                        ax6.set_xlabel('Time (h)')
                        ax6.set_ylabel('Concentration (M)')
                        ax7.legend(['RhlI'])
                        ax7.set_xlabel('Time (h)')
                        ax7.set_ylabel('Concentration (M)')

                        fig8, ax8 = plt.subplots()

                        m[m.Signal=='mCherry in Strain 1'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax8)
                        m[m.Signal=='CFP in Strain 2'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax8)
                        m[m.Signal=='mVenus in Strain 2'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax8)
                        ax8.legend(['mCherry','CFP', 'mVenus'])
                        ax8.set_xlabel('Time (h)')
                        ax8.set_ylabel('Concentration (M)')

                        fig9, ax9 = plt.subplots()

                        m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax9)
                        m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax9)

                        ax9.legend(['Ext C4', 'Ext C14'])
                        ax9.set_xlabel('Time (h)')
                        ax9.set_ylabel('Concentration (M)')

                        # assay.run(substeps=50)
                        # m = assay.measurements
                        # fig8, ax8 = plt.subplots()
                        # m[m.Signal=='mCherry'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax8)
                        # m[m.Signal=='CFP'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax8)
                        # m[m.Signal=='mVenus'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax8)
                        # ax8.legend(['mCherry','CFP', 'mVenus'])
                        # ax8.set_xlabel('Time (h)')
                        # ax8.set_ylabel('Concentration (M)')

